<a href="https://colab.research.google.com/github/advapplab/esgBERT_hf/blob/main/MLM_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install awscli

In [ ]:
import os

!export AWS_SHARED_CREDENTIALS_FILE = /content/awscli.ini

path = "/content/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

In [ ]:
!aws s3 cp s3://esg-bert-pretraining/pretraining_testingdata.csv /content/

In [ ]:
!pip install transformers datasets pandas numpy evaluate

In [ ]:
import datasets
import pandas as pd
input_df = pd.read_csv('pretraining_testingdata.csv', encoding = 'ISO-8859-1')
input_df = input_df[['content']]

raw_datasets = datasets.DatasetDict(
    {
        "evaluate": datasets.Dataset.from_pandas(input_df)
    }
)

In [ ]:
from transformers import BertTokenizer
context_length = 128
model_checkpoint = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_function(texts):
    return tokenizer(texts["content"], padding="max_length", truncation=True) # max_length=context_length

tokenized_datasets = raw_datasets.map(tokenize_function)

  0%|          | 0/300 [00:00<?, ?ex/s]

In [ ]:
tokenized_datasets

DatasetDict({
    evaluate: Dataset({
        features: ['content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 300
    })
})

In [ ]:
eval_dataset = tokenized_datasets['evaluate'].shuffle(seed=42).select(range(256))
eval_dataset

Dataset({
    features: ['content', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 256
})

In [ ]:
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, targets = eval_pred.predictions, eval_pred.label_ids
    maxProbInds = np.argmax(predictions, axis=-1)
    
    mask_pos_preds =  maxProbInds[tuple([targets != -100])]
    mask_pos_targets = targets[tuple([targets != -100])]
    
    # python (no numpy)
    # mask_pos_preds, mask_pos_targets = [], []
    # seq_count, seq_len = targets.shape
    # for s in range(seq_count):
    #     for t in range(seq_len):
    #         if targets[s][t] != -100:
    #             mask_pos_preds.append(maxProbInds[s][t])
    #             mask_pos_targets.append(targets[s][t])
    
    return metric.compute(predictions=mask_pos_preds, references=mask_pos_targets)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: content. If content are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 256
  Batch size = 8


{'eval_loss': 2.558933973312378,
 'eval_accuracy': 0.5535665257262095,
 'eval_runtime': 198.9284,
 'eval_samples_per_second': 1.287,
 'eval_steps_per_second': 0.161}